In [1]:
import os
import pandas as pd

TRAINING_FILE = "train.csv"

def load_data(file, header=True):
    csv_path = os.path.join("", file)
    if header:
        return pd.read_csv(csv_path)
    else:
        return pd.read_csv(csv_path, header=None)


data = load_data(TRAINING_FILE)
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
import pandas as pd

data = load_data(TRAINING_FILE)
def age_filler(data):
    agedata = data.copy()
    
    agedata.drop("Survived", axis=1, inplace=True)
    agedata.drop("Name", axis=1, inplace=True)
    agedata.drop("Ticket", axis=1, inplace=True)
    agedata.drop("Cabin", axis=1, inplace=True)
    agedata.drop("Embarked", axis=1, inplace=True)

    sex_encoder = LabelBinarizer()
    gender_1hot = sex_encoder.fit_transform(agedata["Sex"])
    agedata["Gender"] = gender_1hot
    agedata.drop("Sex", axis=1, inplace=True)
    
    X_ = agedata[agedata["Age"].notnull()]
    Y = X_["Age"].values
    X_ = X_.drop("Age", axis=1)
    X_ = X_.drop("PassengerId", axis=1)
    X = X_[list(X_)].values

    test_X_ = agedata[agedata["Age"].isnull()]
    ids = test_X_["PassengerId"].copy()
    test_X_ = test_X_.drop("Age", axis=1)
    test_X_ = test_X_.drop("PassengerId", axis=1)
    test_X = test_X_[list(test_X_)].values

    scalar = StandardScaler()
    scalar.fit(X)
    X = pd.DataFrame(scalar.transform(X), columns=X_.columns)
    test_X = pd.DataFrame(scalar.transform(test_X), columns=test_X_.columns)

    tree_reg = DecisionTreeRegressor()
    tree_reg.fit(X, Y)
    test_Y = tree_reg.predict(test_X)

    df = pd.DataFrame(columns=["PassengerId", "Age"])
    df["PassengerId"] = ids.values
    df["Age"] = test_Y
    data = pd.merge(left=data, right=df, how='left', on='PassengerId')
    data["Age_x"].fillna(0, inplace=True)
    data["Age_y"].fillna(0, inplace=True)
    data["Age"] = data["Age_x"] + data["Age_y"]
    data.drop("Age_x", axis=1, inplace=True)
    data.drop("Age_y", axis=1, inplace=True)
    return data

data = age_filler(data)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
Age            891 non-null float64
dtypes: float64(2), int64(5), object(5)
memory usage: 90.5+ KB
